- 没有明确需求的用户访问了我们的服务，且服务对用户构成了信息过载，系统通过一定的规则对物品进行排序，并将排在前面的物品展示给用户，这样的系统就是推荐系统


### lambda架构
- 高容错
- 低延迟
- 可扩展

In [1]:
import pandas as pd
import numpy as np
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]
# 用户购买记录数据集
datasets = [
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1],
]

In [2]:
df = pd.DataFrame(data=datasets,index=users,columns=items)
df

,Item A,Item B,Item C,Item D,Item E
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


## 基于用户的协同过滤推荐

- “跟你喜好相似的人喜欢的东西你也很有可能喜欢” ：基于用户的协同过滤推荐（User-based CF）

In [32]:
# 方法一采用皮尔逊相关系数计算

print("用户之间的两两相似度：")
# 直接计算皮尔逊相关系数
# 默认是按列进行计算，因此如果计算用户间的相似度，当前需要进行转置
user_similar = df.T.corr()
print(user_similar.round(4))


用户之间的两两相似度：
        User1   User2   User3   User4   User5
User1  1.0000  0.1667  0.6667 -0.6667 -0.4082
User2  0.1667  1.0000 -0.1667  0.1667 -0.4082
User3  0.6667 -0.1667  1.0000 -1.0000  0.4082
User4 -0.6667  0.1667 -1.0000  1.0000 -0.4082
User5 -0.4082 -0.4082  0.4082 -0.4082  1.0000


In [77]:
# 方法二杰卡德相似系数
from sklearn.metrics import jaccard_similarity_score
# 直接计算某两项的杰卡德相似系数
# 计算Item A 和Item B的相似度
print(jaccard_similarity_score(df["Item A"], df["Item B"]))

0.2


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:660: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


In [36]:
# 方法二

from sklearn.metrics.pairwise import pairwise_distances
# 计算用户间相似度
user_similar = 1 - pairwise_distances(df, metric="hamming")
user_similar = pd.DataFrame(user_similar, columns=users, index=users)
print("用户之间的两两相似度：")
print(user_similar)

用户之间的两两相似度：
       User1  User2  User3  User4  User5
User1    1.0    0.6    0.8    0.2    0.4
User2    0.6    1.0    0.4    0.6    0.4
User3    0.8    0.4    1.0    0.0    0.6
User4    0.2    0.6    0.0    1.0    0.4
User5    0.4    0.4    0.6    0.4    1.0


In [37]:
from pprint import pprint
# 筛选TopN的结果
topN_users = {}
# 遍历每一行数据
for i in user_similar.index:
    # 取出每一列数据，并删除自身，然后排序数据
    _df = user_similar.loc[i].drop([i])
    # print(_df)
    #sort_values 排序 按照相似度降序排列
    _df_sorted = _df.sort_values(ascending=False)
    # print(_df_sorted)
     # 从排序之后的结果中切片 取出前两条（相似度最高的两个）
    top2 = list(_df_sorted.index[:2])
    # print(top2)
    topN_users[i]=top2
print("Top2相似用户:")
pprint(topN_users)


Top2相似用户:
{'User1': ['User3', 'User2'],
 'User2': ['User4', 'User1'],
 'User3': ['User1', 'User5'],
 'User4': ['User2', 'User5'],
 'User5': ['User3', 'User4']}


In [38]:
# 计算推荐的结果
rs_results = {}
# 遍历所有最相似的用户
for user,sim_users in topN_users.items():
    rs_result = set()
    for sim_user in sim_users:
        # 构建初始推荐结果
        rs_result = rs_result.union(set(df.loc[sim_user].replace(0,np.nan).dropna().index))
    # 过滤掉已经购买过的物品
    rs_result -= set(df.loc[user].replace(0,np.nan).dropna().index)
    rs_results[user]=rs_result
    
print("最终推荐结果")
pprint(rs_results)

最终推荐结果
{'User1': {'Item E'},
 'User2': {'Item C', 'Item B'},
 'User3': {'Item D', 'Item B', 'Item E'},
 'User4': {'Item C', 'Item A'},
 'User5': {'Item D'}}


最终推荐结果
{'User1': {'Item E'},
 'User2': {'Item C', 'Item B'},
 'User3': {'Item D', 'Item B', 'Item E'},
 'User4': {'Item C', 'Item A'},
 'User5': {'Item D'}}

## 基于物品的协同过滤推荐

- 跟你喜欢的东西相似的东西你也很有可能喜欢 ”：基于物品的协同过滤推荐（Item-based CF）

In [7]:
import pandas as pd
import numpy as np
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]
# 用户购买记录数据集
datasets = [
    [1,0,1,1,0],
    [1,0,0,1,1],
    [1,0,1,0,0],
    [0,1,0,1,1],
    [1,1,1,0,1],
]

In [8]:
df = pd.DataFrame(data=datasets,index=users,columns=items)
df

,Item A,Item B,Item C,Item D,Item E
User1,1,0,1,1,0
User2,1,0,0,1,1
User3,1,0,1,0,0
User4,0,1,0,1,1
User5,1,1,1,0,1


In [26]:
# 方法一
print("物品之间的两两相似度：")
item_similar = df.corr()
print(item_similar.round(4))

物品之间的两两相似度：
        Item A  Item B  Item C  Item D  Item E
Item A  1.0000 -0.6124  0.6124 -0.4082 -0.4082
Item B -0.6124  1.0000 -0.1667 -0.1667  0.6667
Item C  0.6124 -0.1667  1.0000 -0.6667 -0.6667
Item D -0.4082 -0.1667 -0.6667  1.0000  0.1667
Item E -0.4082  0.6667 -0.6667  0.1667  1.0000


In [29]:
# 方法二

from sklearn.metrics.pairwise import pairwise_distances
# 计算用户间相似度
item_similar = 1 - pairwise_distances(df.T, metric="hamming")
item_similar = pd.DataFrame(item_similar, columns=items, index=items)
print("物品之间的两两相似度：")
print(item_similar)

物品之间的两两相似度：
        Item A  Item B  Item C  Item D  Item E
Item A     1.0     0.2     0.8     0.4     0.4
Item B     0.2     1.0     0.4     0.4     0.8
Item C     0.8     0.4     1.0     0.2     0.2
Item D     0.4     0.4     0.2     1.0     0.6
Item E     0.4     0.8     0.2     0.6     1.0


In [30]:
# 找出相似度高的物品
topN_items={}
for i in item_similar.index:
    _df = item_similar.loc[i].drop([i])
    # print(_df)
    # 安相似度降序排列
    df_sorted = _df.sort_values(ascending=False)
    #  截取前两个
    top2 = list(df_sorted.index[:2])
    topN_items[i] = top2
    
print('Top2相似物品')
print(topN_items)

Top2相似物品
{'Item A': ['Item C', 'Item E'], 'Item B': ['Item E', 'Item D'], 'Item C': ['Item A', 'Item B'], 'Item D': ['Item E', 'Item B'], 'Item E': ['Item B', 'Item D']}


In [31]:
# 根据相似度物品进行推荐
rs_results = {}

for user in df.index:
    rs_result = set()
    for item in df.loc[user].replace(0,np.nan).dropna().index:
        # 根据每个物品找出最相似的TOP_N物品，构建初始推荐结果
        rs_result = rs_result.union(set(topN_items[item]))
    # 过滤掉用户已购买的物品
    rs_result -= set(df.loc[user].replace(0,np.nan).dropna().index)
    rs_results[user]=rs_result
print("最终推荐的结果为")
pprint(rs_results)

最终推荐的结果为
{'User1': {'Item B', 'Item E'},
 'User2': {'Item C', 'Item B'},
 'User3': {'Item B', 'Item E'},
 'User4': set(),
 'User5': {'Item D'}}


最终推荐的结果为
{'User1': {'Item B', 'Item E'},
 'User2': {'Item C', 'Item B'},
 'User3': {'Item B', 'Item E'},
 'User4': set(),
 'User5': {'Item D'}}

## 使用协同过滤推荐算法对用户进行评分预测

In [40]:
users = ["User1", "User2", "User3", "User4", "User5"]
items = ["Item A", "Item B", "Item C", "Item D", "Item E"]
# 用户购买记录数据集
datasets = [
    [5,3,4,4,None],
    [3,1,2,3,3],
    [4,3,4,3,5],
    [3,3,1,5,4],
    [1,5,5,2,1],
]

In [41]:
# 使用皮尔逊相关系数计算来计算
df = pd.DataFrame(datasets,
                  columns=items,
                  index=users)

print("用户之间的两两相似度：")
# 直接计算皮尔逊相关系数
# 默认是按列进行计算，因此如果计算用户间的相似度，当前需要进行转置
user_similar = df.T.corr()
print(user_similar.round(4))

print("物品之间的两两相似度：")
item_similar = df.corr()
print(item_similar.round(4))

用户之间的两两相似度：
        User1   User2   User3   User4   User5
User1  1.0000  0.8528  0.7071  0.0000 -0.7921
User2  0.8528  1.0000  0.4677  0.4900 -0.9001
User3  0.7071  0.4677  1.0000 -0.1612 -0.4666
User4  0.0000  0.4900 -0.1612  1.0000 -0.6415
User5 -0.7921 -0.9001 -0.4666 -0.6415  1.0000
物品之间的两两相似度：
        Item A  Item B  Item C  Item D  Item E
Item A  1.0000 -0.4767 -0.1231  0.5322  0.9695
Item B -0.4767  1.0000  0.6455 -0.3101 -0.4781
Item C -0.1231  0.6455  1.0000 -0.7206 -0.4276
Item D  0.5322 -0.3101 -0.7206  1.0000  0.5817
Item E  0.9695 -0.4781 -0.4276  0.5817  1.0000
